In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sys
sys.path.append("/home/maddie/SmartIoT-2025/GANs models/")
from ad_utils import *

cuda0 = torch.device("cuda:0")
cuda1 = torch.device("cuda:1")
device = cuda1
print(torch.cuda.get_device_name(device) if torch.cuda.is_available() else "No GPU available")

data = np.load("hvcm/RFQ.npy", allow_pickle=True)
label = np.load("hvcm/RFQ_labels.npy", allow_pickle=True)
label = label[:, 1]  # Assuming the second column is the label
label = (label == "Fault").astype(int)  # Convert to binary labels
print(data.shape, label.shape)
scaler = StandardScaler()
data = scaler.fit_transform(data.reshape(-1, data.shape[-1])).reshape(data.shape)
normal_data = data[label == 0]
faulty_data = data[label == 1]

normal_label = label[label == 0]
faulty_label = label[label == 1]

X_train_normal, X_test_normal, y_train_normal, y_test_normal = train_test_split(normal_data, normal_label, test_size=0.2, random_state=42, shuffle=True)
X_train_faulty, X_test_faulty, y_train_faulty, y_test_faulty = train_test_split(faulty_data, faulty_label, test_size=0.2, random_state=42, shuffle=True)

labels = np.concatenate((y_test_normal, y_test_faulty))

NVIDIA A30
(872, 4500, 14) (872,)


In [3]:
run_pipeline_with_cv(X_train_normal, X_test_normal, X_test_faulty, 
                    device=device, batch_size=64, num_epochs=20, cv_folds=5)

Extracting features from all samples...


/home/maddie/SmartIoT-2025/.venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/maddie/SmartIoT-2025/.venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Starting 5-fold cross-validation...

Fold 1/5
------------------------------
Train normal samples: 110
Test samples: 35 (28 normal, 7 anomaly)
  Epoch 1/20, Loss: 0.282764
  Epoch 6/20, Loss: 0.256913
  Epoch 11/20, Loss: 0.082879
  Epoch 16/20, Loss: 0.026469
  Epoch 20/20, Loss: 0.020862
Results: Acc=0.9143, Prec=1.0000, Rec=0.5714, F1=0.7273
Optimal threshold: 0.027954

Fold 2/5
------------------------------
Train normal samples: 110
Test samples: 35 (28 normal, 7 anomaly)
  Epoch 1/20, Loss: 0.282949
  Epoch 6/20, Loss: 0.261869
  Epoch 11/20, Loss: 0.093520
  Epoch 16/20, Loss: 0.028437
  Epoch 20/20, Loss: 0.021355
Results: Acc=0.8571, Prec=1.0000, Rec=0.2857, F1=0.4444
Optimal threshold: 0.024645

Fold 3/5
------------------------------
Train normal samples: 110
Test samples: 35 (28 normal, 7 anomaly)
  Epoch 1/20, Loss: 0.282900
  Epoch 6/20, Loss: 0.233402
  Epoch 11/20, Loss: 0.063093
  Epoch 16/20, Loss: 0.024510
  Epoch 20/20, Loss: 0.020550
Results: Acc=0.9429, Prec=1.00

{'fold_results': [{'fold': 1,
   'train_samples': 110,
   'test_samples': 35,
   'final_train_loss': 0.020862361924214798,
   'optimal_threshold': 0.02795434579478972,
   'accuracy': 0.9142857142857143,
   'precision': 1.0,
   'recall': 0.5714285714285714,
   'f1': 0.7272727272727273},
  {'fold': 2,
   'train_samples': 110,
   'test_samples': 35,
   'final_train_loss': 0.021354577419432728,
   'optimal_threshold': 0.02464501288804141,
   'accuracy': 0.8571428571428571,
   'precision': 1.0,
   'recall': 0.2857142857142857,
   'f1': 0.4444444444444445},
  {'fold': 3,
   'train_samples': 110,
   'test_samples': 35,
   'final_train_loss': 0.020550183443860573,
   'optimal_threshold': 0.024018629200079224,
   'accuracy': 0.9428571428571428,
   'precision': 1.0,
   'recall': 0.7142857142857143,
   'f1': 0.8333333333333333},
  {'fold': 4,
   'train_samples': 111,
   'test_samples': 35,
   'final_train_loss': 0.020483428612351418,
   'optimal_threshold': 0.02502757784995166,
   'accuracy': 0.9